# DB2-Salesforce connector: Citation mapping to users

In [1]:
# API settings
bulk_api_threshold = 100 # if more records than this, use Bulk API instead
disallow_bulk_api = False # if Bulk API is allowed

api_url = '/services/data/v43.0/sobjects'
external_id = 'Name'
object_id = 'contact_citation_asso__c'

# login parameters to be handled by Papermill
sf_login_params = None 
sql_login_params = None

In [3]:
# Parameters
sql_login_params = {"username": "wang159_ro", "password": "napoleon0"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username": "wang159-4j1v@force.com",
    "password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",
}


In [4]:
#%load_ext autoreload

#%autoreload 2
    
import pandas as pd
import time
import datetime

from DB2SalesforceAPI import DB2SalesforceAPI

## Obtain tool information from DB2 

In [5]:
# connect with DB2
import sqlalchemy as sql

engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub' \
                                               %(sql_login_params['username'], sql_login_params['password']))

# Citations
sql_query = "select * from jos_citations"

citations_df = pd.read_sql_query(sql_query, engine)

# Citation authors
sql_query = "select * from jos_citations_authors;"

authors_df = pd.read_sql_query(sql_query, engine)

In [6]:
# combining authors and citations
ac_df = pd.merge(citations_df[['id']], authors_df[['cid', 'uidNumber','id']], how='inner',\
                 left_on='id', right_on='cid', suffixes=['_c', '_a'])\
                .drop(columns='id_c')

ac_df.rename(columns={'cid':'citation_ID', 'uidNumber':'nanohub_user_id', 'id_a':'author_id'}, inplace=True)

# display
ac_df.head(3)

,citation_ID,nanohub_user_id,author_id
0,10000003,10676,4613
1,10000003,0,4614
2,10000003,37033,4615


## Obtain Salesforce IDs

In [7]:
# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [8]:
# query the Salesforce IDs for contacts and citations. when updating junction objects, these IDs must be used

# get Salesforce ID for contacts
sf_userID_df = db_s.query_data('SELECT Id, nanoHUB_user_ID__c FROM Contact where nanoHUB_user_ID__c != NULL')

sf_userID_df['nanoHUB_user_ID__c'] = sf_userID_df['nanoHUB_user_ID__c'].astype('int')

[Success] Bulk job creation successful. Job ID = 7505w00000M1rJWAAZ
{"id":"7505w00000M1rJWAAZ","operation":"query","object":"Contact","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-05-15T00:46:57.000+0000","systemModstamp":"2020-05-15T00:46:58.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000M1rJWAAZ","operation":"query","object":"Contact","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-05-15T00:46:57.000+0000","systemModstamp":"2020-05-15T00:46:58.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000M1rJWAAZ","operation":"query","object":"Contact","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-05-15T00:46:57

In [9]:
# get Salesforce ID for citations
sf_citationID_df = db_s.query_data('SELECT Id, Record_ID__c FROM nanoHUB_citations__c')

sf_citationID_df['Record_ID__c'] = sf_citationID_df['Record_ID__c'].astype('int')

[Success] Bulk job creation successful. Job ID = 7505w00000M1rJlAAJ
{"id":"7505w00000M1rJlAAJ","operation":"query","object":"nanoHUB_citations__c","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-05-15T00:47:21.000+0000","systemModstamp":"2020-05-15T00:47:21.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000M1rJlAAJ","operation":"query","object":"nanoHUB_citations__c","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-05-15T00:47:21.000+0000","systemModstamp":"2020-05-15T00:47:22.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":2449,"retries":0,"totalProcessingTime":280}
[Success] Bulk job completed successfully.


## Match data with Salesforce format

In [10]:
# valid citation - nanohub user links
ac_tolink_df = ac_df[ac_df.nanohub_user_id != 0]

display(ac_tolink_df.head(2))
display(sf_citationID_df.head(2))
display(sf_userID_df.head(2))

,citation_ID,nanohub_user_id,author_id
0,10000003,10676,4613
2,10000003,37033,4615


,Id,Record_ID__c
0,a0t5w000008p7pSAAQ,1279
1,a0t5w000008p7pTAAQ,1535


,Id,nanoHUB_user_ID__c
0,0035w000031Vsp1AAC,998
1,0035w000031Vsp2AAC,1683


In [11]:
# merge SF citation and contact IDs into user-citation DF
ac_tolink_df = pd.merge(ac_tolink_df, sf_citationID_df, how='inner', left_on='citation_ID', right_on='Record_ID__c')\
                           .rename(columns={'Id':'SF_ID_citation'})
                                            
ac_tolink_df = pd.merge(ac_tolink_df, sf_userID_df, how='inner', left_on='nanohub_user_id', right_on='nanoHUB_user_ID__c')\
                           .rename(columns={'Id':'SF_ID_contact'})

# display
ac_tolink_df.head(2)

,citation_ID,nanohub_user_id,author_id,SF_ID_citation,Record_ID__c,SF_ID_contact,nanoHUB_user_ID__c
0,1279,30551,11732936,a0t5w000008p7pSAAQ,1279,0035w000034JLpOAAW,30551
1,3064,30551,11732648,a0t5w000008p7q1AAA,3064,0035w000034JLpOAAW,30551


In [12]:
# create a new column for object name
ac_tolink_df['Name'] = ac_tolink_df.apply(lambda x: '%d_%d'%(x.nanoHUB_user_ID__c, x.Record_ID__c), axis=1)

In [13]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here
df_sf['Name']         = ac_tolink_df['Name']
df_sf['Contact__c'] = ac_tolink_df['SF_ID_contact']
df_sf['Citation__c'] = ac_tolink_df['SF_ID_citation']

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
Name,30551_1279,30551_3064
Contact__c,0035w000034JLpOAAW,0035w000034JLpOAAW
Citation__c,a0t5w000008p7pSAAQ,a0t5w000008p7q1AAA


## To Salesforce Sales Cloud CRM

In [14]:
# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

# specify Salesforce object ID and external ID
db_s.object_id = 'contact_citation_asso__c'
db_s.external_id = 'Name'

Obtained Salesforce access token ...... True


In [15]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000M1rBsAAJ
[Success] CSV upload successful. Job ID = 7505w00000M1rBsAAJ
[Success] Closing job successful. Job ID = 7505w00000M1rBsAAJ


In [17]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_status())

[{'errorCode': 'INVALID_SESSION_ID', 'message': 'Session expired or invalid'}]
